In [1]:
import MetaTrader5 as mt5
import numpy as np
import pandas as pd
from datetime import datetime
from  backtest import Backtester,  get_ohlc_history, create_price_fig, evaluate_backtest

In [2]:
# Conecte mt5
mt5.initialize()
# logine mt5
login = 52185665
password = '3cAJz$AQiKwMq0'
server = 'ICMarketsSC-Demo'

mt5.login(login,password,server)

True

In [3]:
# this function retreives olhc data from mt5 account and return a data frame
def get_ohlc(symbol, timeframe, start_datetime, end_datetime):
    ohlc = mt5.copy_rates_range(symbol, timeframe, start_datetime, end_datetime)
    df = pd.DataFrame(ohlc)
    df['time'] = pd.to_datetime(df['time'], unit= 's')#.dt.tz_localize('UTC')
    return df

In [4]:
# 1 minut df
symbol = 'EURUSD'
pos_size = 1
timeframe = mt5.TIMEFRAME_M1
start_datetime = datetime(2025,7,1)
end_datetime = datetime.now()
df = get_ohlc(symbol, timeframe, start_datetime, end_datetime)
#df.set_index("time", inplace=True)
#df = df.between_time("13:00", "16:00")
df

,time,open,high,low,close,tick_volume,spread,real_volume
0,2025-06-30 23:00:00,1.17797,1.17798,1.17774,1.17797,74,0,0
1,2025-06-30 23:01:00,1.17797,1.17807,1.17794,1.17795,49,0,0
2,2025-06-30 23:02:00,1.17795,1.17801,1.17795,1.17799,51,0,0
3,2025-06-30 23:03:00,1.17799,1.17800,1.17791,1.17797,26,0,0
4,2025-06-30 23:04:00,1.17797,1.17801,1.17791,1.17791,26,0,0
...,...,...,...,...,...,...,...,...
52661,2025-08-20 13:34:00,1.16567,1.16574,1.16529,1.16530,67,0,0
52662,2025-08-20 13:35:00,1.16530,1.16534,1.16523,1.16524,46,0,0
52663,2025-08-20 13:36:00,1.16524,1.16529,1.16518,1.16519,47,0,0
52664,2025-08-20 13:37:00,1.16519,1.16521,1.16512,1.16514,70,0,0


In [5]:
# === Calculate indicators ===
df["atr"] = df["high"].rolling(14).max() - df["low"].rolling(14).min()
df["vwap"] = (df["close"] * df["tick_volume"]).cumsum() / df["tick_volume"].cumsum()
df["volume_spike"] = df["tick_volume"] > df["tick_volume"].rolling(10).mean() * 1.5

# === Signal Generation ===
def generate_signals(df):
    signals = []
    for i in range(14, len(df)):
        row = df.iloc[i]
        prev = df.iloc[i-1]

        cond1 = row["close"] < row["vwap"] - 2 * row["atr"]
        cond2 = row["spread"] < 0.0002
        cond3 = row["volume_spike"]
        cond4 = row["close"] > row["open"] and prev["close"] < prev["open"]

        if cond1 and cond2 and cond3 and cond4: 
            signals.append(i)
    return signals

signals = generate_signals(df)

# === Backtest ===
results = []
rr = 3  # risk-reward ratio

for i in signals:
    entry_price = df.iloc[i]["close"]
    sl = entry_price - 0.5 * df.iloc[i]["atr"]
    tp = entry_price + rr * (entry_price - sl)

    for j in range(i+1, min(i+60, len(df))):  # look ahead for next ~2 hours
        high = df.iloc[j]["high"]
        low = df.iloc[j]["low"]
        if low <= sl:
            results.append(("loss", entry_price, sl))
            break
        elif high >= tp:
            results.append(("win", entry_price, tp))
            break
    else:
        # time-based exit
        close_exit = df.iloc[i+60]["close"]
        if close_exit > entry_price:
            results.append(("win", entry_price, close_exit))
        else:
            results.append(("loss", entry_price, close_exit))
# === Metrics ===
wins = [r for r in results if r[0] == "win"]
losses = [r for r in results if r[0] == "loss"]
win_rate = len(wins) / len(results)
r_r_ratio = np.mean([abs(w[2]-w[1]) for w in wins]) / np.mean([abs(l[2]-l[1]) for l in losses])


entry_price = df.iloc[i]["close"]
df['sl'] = entry_price - 0.5 * df.iloc[i]["atr"]
df['tp'] = entry_price + rr * (entry_price - sl)

print(f"Total trades: {len(results)}")
print(f"Win rate: {win_rate:.2%}")
print(f"Risk-Reward Ratio: {r_r_ratio:.2f}")
print(results)


Total trades: 427
Win rate: 28.81%
Risk-Reward Ratio: 2.55
[('win', np.float64(1.17683), np.float64(1.1776850000000003)), ('loss', np.float64(1.17714), np.float64(1.1768100000000001)), ('loss', np.float64(1.17751), np.float64(1.1772)), ('loss', np.float64(1.17733), np.float64(1.1770550000000002)), ('loss', np.float64(1.17525), np.float64(1.17515)), ('loss', np.float64(1.17527), np.float64(1.1751200000000002)), ('loss', np.float64(1.1751800000000001), np.float64(1.1750350000000003)), ('loss', np.float64(1.17516), np.float64(1.17495)), ('win', np.float64(1.1749100000000001), np.float64(1.1751649999999998)), ('loss', np.float64(1.17522), np.float64(1.17502)), ('loss', np.float64(1.17492), np.float64(1.1746949999999998)), ('win', np.float64(1.17485), np.float64(1.175585)), ('loss', np.float64(1.17576), np.float64(1.1756799999999998)), ('win', np.float64(1.17565), np.float64(1.17591)), ('win', np.float64(1.17563), np.float64(1.1758849999999996)), ('loss', np.float64(1.17595), np.float64(1.1

In [6]:
prev = df.iloc[i-1]
def entring_stoploss_profit(row) :
    hour = row['time'].hour
    #idx = row.name
    #if idx == 0:
        #return None
    #prev = df.iloc[idx - 1]

    if (
        row["close"] < row["vwap"] - 2 * row["atr"]
        and row["spread"] < 0.0002
        and row["volume_spike"]
        and row["close"] > row["open"]
        and prev["close"] < prev["open"]
        #and (hour < 11 or hour > 15)
    ):
        return "buy"
    return None
df["signal"] = df.apply(entring_stoploss_profit, axis=1)
df.copy().dropna()

,time,open,high,low,close,tick_volume,spread,real_volume,atr,vwap,volume_spike,sl,tp,signal
1259,2025-07-01 20:00:00,1.17654,1.17690,1.17654,1.17687,96,0,0,0.00075,1.179205,True,1.163475,1.164735,buy
1357,2025-07-01 21:38:00,1.17772,1.17779,1.17771,1.17777,52,0,0,0.00025,1.179125,True,1.163475,1.164735,buy
2249,2025-07-02 12:31:00,1.17680,1.17687,1.17679,1.17683,75,0,0,0.00057,1.179102,True,1.163475,1.164735,buy
2266,2025-07-02 12:48:00,1.17708,1.17719,1.17708,1.17714,64,0,0,0.00066,1.179087,True,1.163475,1.164735,buy
2303,2025-07-02 13:25:00,1.17749,1.17759,1.17743,1.17751,56,0,0,0.00062,1.179062,True,1.163475,1.164735,buy
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52323,2025-08-20 07:56:00,1.16349,1.16354,1.16346,1.16351,41,0,0,0.00020,1.165661,True,1.163475,1.164735,buy
52372,2025-08-20 08:45:00,1.16374,1.16388,1.16374,1.16388,42,0,0,0.00023,1.165660,True,1.163475,1.164735,buy
52385,2025-08-20 08:58:00,1.16377,1.16400,1.16377,1.16398,54,0,0,0.00037,1.165659,True,1.163475,1.164735,buy
52387,2025-08-20 09:00:00,1.16393,1.16422,1.16391,1.16400,105,0,0,0.00057,1.165659,True,1.163475,1.164735,buy


In [7]:

# create trade logic
def on_bar(data, trades, orders):
    open_trades = trades[trades['state'] == 'open']
    num_open_trades = open_trades.shape[0]
    account_balance = 10000  # your account in USD
    risk_percent = 0.20  # 2%
    pip_value_per_lot = 10  # on EURUSD, 1 lot = $10 per pip
    risk_amount = account_balance * risk_percent
    # entry signal
    if data['signal'] == 'buy' :#and not num_open_trades:
        volume = 100000 #risk_amount / (stop_loss_pips * pip_value_per_lot)
        orders.open_trade(symbol, volume, 'buy')
    
    elif data['signal'] == 'sell' and not num_open_trades:
        volume = 100000 #risk_amount / (stop_loss_pips * pip_value_per_lot)
        orders.open_trade(symbol, volume, 'sell')
        
# exit signal
    if num_open_trades:
        trade = open_trades.iloc[0]

        if trade['order_type'] == 'buy' and data["low"] <= data["sl"] or data["high"] >= data["tp"]: 
            orders.close_trade(trade)
        elif trade['order_type'] == 'sell'  :
            orders.close_trade(trade)

In [8]:
# backtest parameters
starting_balance = 10000
currency = 'USD'
exchange_rate = 1
commission = -7 / 100000

# backtest
bt = Backtester()
bt.set_starting_balance(starting_balance, currency=currency)
bt.set_exchange_rate(exchange_rate)
bt.set_commission(commission)

bt.set_historical_data(df)
bt.set_on_bar(on_bar)

get_ohlc_history(symbol, timeframe, start_datetime, end_datetime)
bt.run_backtest()

bt.trades


,state,symbol,order_type,volume,open_time,open_price,close_time,close_price,sl,tp,info,profit,commission,profit_net,profit_cumulative,balance
0,closed,EURUSD,buy,100000,2025-07-01 20:00:00,1.17654,2025-07-01 20:01:00,1.17686,0,0,{},32.0,-7.0,25.0,25.0,10025.0
1,closed,EURUSD,buy,100000,2025-07-01 21:38:00,1.17772,2025-07-01 21:39:00,1.17777,0,0,{},5.0,-7.0,-2.0,23.0,10023.0
2,closed,EURUSD,buy,100000,2025-07-02 12:31:00,1.1768,2025-07-02 12:32:00,1.17683,0,0,{},3.0,-7.0,-4.0,19.0,10019.0
3,closed,EURUSD,buy,100000,2025-07-02 12:48:00,1.17708,2025-07-02 12:49:00,1.17714,0,0,{},6.0,-7.0,-1.0,18.0,10018.0
4,closed,EURUSD,buy,100000,2025-07-02 13:25:00,1.17749,2025-07-02 13:26:00,1.17751,0,0,{},2.0,-7.0,-5.0,13.0,10013.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1002,closed,EURUSD,buy,100000,2025-08-20 07:56:00,1.16349,2025-08-20 07:57:00,1.16351,0,0,{},2.0,-7.0,-5.0,6840.0,16840.0
1003,closed,EURUSD,buy,100000,2025-08-20 08:45:00,1.16374,2025-08-20 09:03:00,1.16365,0,0,{},-9.0,-7.0,-16.0,6824.0,16824.0
1004,closed,EURUSD,buy,100000,2025-08-20 08:58:00,1.16377,2025-08-20 09:04:00,1.16341,0,0,{},-36.0,-7.0,-43.0,6781.0,16781.0
1005,closed,EURUSD,buy,100000,2025-08-20 09:00:00,1.16393,2025-08-20 09:05:00,1.1633,0,0,{},-63.0,-7.0,-70.0,6711.0,16711.0


In [9]:
mm = bt.plot_balance()
display(mm)

In [10]:
evaluate_backtest(bt.trades)

Daily Drawdown : -330.00
Max Drawdown (portfolio): -330.00
worst trades (trade MAE): [np.float64(0.0), np.float64(-0.9999999999843467), np.float64(-7.0000000000014495), np.float64(0.0), np.float64(-5.999999999994898), np.float64(-28.000000000005798), np.float64(0.0), np.float64(-7.999999999985796), np.float64(-1.0000000000065512), np.float64(-11.999999999989797), np.float64(0.0), np.float64(-0.9999999999843467), np.float64(-1.0000000000065512), np.float64(-1.0000000000065512), np.float64(-14.999999999987246), np.float64(-1.0000000000065512), np.float64(0.0), np.float64(-0.9999999999843467), np.float64(0.0), np.float64(-2.0000000000131024), np.float64(0.0), np.float64(0.0), np.float64(-2.999999999997449), np.float64(-7.999999999985796), np.float64(0.0), np.float64(0.0), np.float64(-4.000000000004), np.float64(0.0), np.float64(-7.0000000000014495), np.float64(0.0), np.float64(0.0), np.float64(-2.999999999997449), np.float64(-4.000000000004), np.float64(-1.0000000000065512), np.float64(-1

,order_type,profit
0,buy,13786.0


,state,symbol,order_type,volume,open_time,open_price,close_time,close_price,sl,tp,...,profit,commission,profit_net,profit_cumulative,balance,date,time,intrabar_drawdown,current_max,drawdown
0,closed,EURUSD,buy,100000,2025-07-01 20:00:00,1.17654,2025-07-01 20:01:00,1.17686,0,0,...,32.0,-7.0,25.0,25.0,10025.0,2025-07-01,2025-07-01 20:01:00,0.0,25.0,0.0
1,closed,EURUSD,buy,100000,2025-07-01 21:38:00,1.17772,2025-07-01 21:39:00,1.17777,0,0,...,5.0,-7.0,-2.0,23.0,10023.0,2025-07-01,2025-07-01 21:39:00,-1.0,25.0,-2.0
2,closed,EURUSD,buy,100000,2025-07-02 12:31:00,1.1768,2025-07-02 12:32:00,1.17683,0,0,...,3.0,-7.0,-4.0,19.0,10019.0,2025-07-02,2025-07-02 12:32:00,-7.0,25.0,-6.0
3,closed,EURUSD,buy,100000,2025-07-02 12:48:00,1.17708,2025-07-02 12:49:00,1.17714,0,0,...,6.0,-7.0,-1.0,18.0,10018.0,2025-07-02,2025-07-02 12:49:00,0.0,25.0,-7.0
4,closed,EURUSD,buy,100000,2025-07-02 13:25:00,1.17749,2025-07-02 13:26:00,1.17751,0,0,...,2.0,-7.0,-5.0,13.0,10013.0,2025-07-02,2025-07-02 13:26:00,-6.0,25.0,-12.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
996,closed,EURUSD,buy,100000,2025-08-20 05:55:00,1.16234,2025-08-20 05:56:00,1.1624,0,0,...,6.0,-7.0,-1.0,6896.0,16896.0,2025-08-20,2025-08-20 05:56:00,-7.0,7123.0,-227.0
997,closed,EURUSD,buy,100000,2025-08-20 06:27:00,1.16334,2025-08-20 06:28:00,1.16339,0,0,...,5.0,-7.0,-2.0,6894.0,16894.0,2025-08-20,2025-08-20 06:28:00,-1.0,7123.0,-229.0
1000,closed,EURUSD,buy,100000,2025-08-20 07:18:00,1.16356,2025-08-20 07:23:00,1.16357,0,0,...,1.0,-7.0,-6.0,6854.0,16854.0,2025-08-20,2025-08-20 07:23:00,-13.0,7123.0,-269.0
1002,closed,EURUSD,buy,100000,2025-08-20 07:56:00,1.16349,2025-08-20 07:57:00,1.16351,0,0,...,2.0,-7.0,-5.0,6840.0,16840.0,2025-08-20,2025-08-20 07:57:00,-4.0,7123.0,-283.0


,state,symbol,order_type,volume,open_time,open_price,close_time,close_price,sl,tp,...,profit,commission,profit_net,profit_cumulative,balance,date,time,intrabar_drawdown,current_max,drawdown
164,closed,EURUSD,buy,100000,2025-07-09 18:38:00,1.17082,2025-07-09 18:39:00,1.17081,0,0,...,-1.0,-7.0,-8.0,854.0,10854.0,2025-07-09,2025-07-09 18:39:00,-11.0,862.0,-8.0
205,closed,EURUSD,buy,100000,2025-07-10 08:59:00,1.17391,2025-07-10 09:00:00,1.1739,0,0,...,-1.0,-7.0,-8.0,1054.0,11054.0,2025-07-10,2025-07-10 09:00:00,-22.0,1068.0,-14.0
457,closed,EURUSD,buy,100000,2025-07-16 23:12:00,1.1639,2025-07-17 00:00:00,1.16368,0,0,...,-22.0,-7.0,-29.0,2439.0,12439.0,2025-07-16,2025-07-17 00:00:00,-165.0,2468.0,-29.0
458,closed,EURUSD,buy,100000,2025-07-16 23:31:00,1.16389,2025-07-17 00:01:00,1.16347,0,0,...,-42.0,-7.0,-49.0,2390.0,12390.0,2025-07-16,2025-07-17 00:01:00,-164.0,2468.0,-78.0
459,closed,EURUSD,buy,100000,2025-07-17 01:01:00,1.16384,2025-07-17 02:51:00,1.16351,0,0,...,-33.0,-7.0,-40.0,2350.0,12350.0,2025-07-17,2025-07-17 02:51:00,-42.0,2468.0,-118.0
460,closed,EURUSD,buy,100000,2025-07-17 01:11:00,1.16407,2025-07-17 02:52:00,1.16352,0,0,...,-55.0,-7.0,-62.0,2288.0,12288.0,2025-07-17,2025-07-17 02:52:00,-74.0,2468.0,-180.0
461,closed,EURUSD,buy,100000,2025-07-17 01:46:00,1.1636,2025-07-17 02:53:00,1.16335,0,0,...,-25.0,-7.0,-32.0,2256.0,12256.0,2025-07-17,2025-07-17 02:53:00,-27.0,2468.0,-212.0
462,closed,EURUSD,buy,100000,2025-07-17 02:00:00,1.16363,2025-07-17 02:54:00,1.16339,0,0,...,-24.0,-7.0,-31.0,2225.0,12225.0,2025-07-17,2025-07-17 02:54:00,-30.0,2468.0,-243.0
463,closed,EURUSD,buy,100000,2025-07-17 02:08:00,1.16358,2025-07-17 02:55:00,1.16347,0,0,...,-11.0,-7.0,-18.0,2207.0,12207.0,2025-07-17,2025-07-17 02:55:00,-25.0,2468.0,-261.0
464,closed,EURUSD,buy,100000,2025-07-17 02:25:00,1.16393,2025-07-17 02:56:00,1.16345,0,0,...,-48.0,-7.0,-55.0,2152.0,12152.0,2025-07-17,2025-07-17 02:56:00,-60.0,2468.0,-316.0


,state,symbol,order_type,volume,open_time,open_price,close_time,close_price,sl,tp,...,balance,date,time,intrabar_drawdown,current_max,drawdown,dayofweek,hourofday,month,year
0,closed,EURUSD,buy,100000,2025-07-01 20:00:00,1.17654,2025-07-01 20:01:00,1.17686,0,0,...,10025.0,2025-07-01,2025-07-01 20:01:00,0.0,25.0,0.0,1,20,2025-07,2025
1,closed,EURUSD,buy,100000,2025-07-01 21:38:00,1.17772,2025-07-01 21:39:00,1.17777,0,0,...,10023.0,2025-07-01,2025-07-01 21:39:00,-1.0,25.0,-2.0,1,21,2025-07,2025
2,closed,EURUSD,buy,100000,2025-07-02 12:31:00,1.1768,2025-07-02 12:32:00,1.17683,0,0,...,10019.0,2025-07-02,2025-07-02 12:32:00,-7.0,25.0,-6.0,2,12,2025-07,2025
3,closed,EURUSD,buy,100000,2025-07-02 12:48:00,1.17708,2025-07-02 12:49:00,1.17714,0,0,...,10018.0,2025-07-02,2025-07-02 12:49:00,0.0,25.0,-7.0,2,12,2025-07,2025
4,closed,EURUSD,buy,100000,2025-07-02 13:25:00,1.17749,2025-07-02 13:26:00,1.17751,0,0,...,10013.0,2025-07-02,2025-07-02 13:26:00,-6.0,25.0,-12.0,2,13,2025-07,2025
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1002,closed,EURUSD,buy,100000,2025-08-20 07:56:00,1.16349,2025-08-20 07:57:00,1.16351,0,0,...,16840.0,2025-08-20,2025-08-20 07:57:00,-4.0,7123.0,-283.0,2,7,2025-08,2025
1003,closed,EURUSD,buy,100000,2025-08-20 08:45:00,1.16374,2025-08-20 09:03:00,1.16365,0,0,...,16824.0,2025-08-20,2025-08-20 09:03:00,-42.0,7123.0,-299.0,2,8,2025-08,2025
1004,closed,EURUSD,buy,100000,2025-08-20 08:58:00,1.16377,2025-08-20 09:04:00,1.16341,0,0,...,16781.0,2025-08-20,2025-08-20 09:04:00,-59.0,7123.0,-342.0,2,8,2025-08,2025
1005,closed,EURUSD,buy,100000,2025-08-20 09:00:00,1.16393,2025-08-20 09:05:00,1.1633,0,0,...,16711.0,2025-08-20,2025-08-20 09:05:00,-94.0,7123.0,-412.0,2,9,2025-08,2025


max_drawdown -412.0
